In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input

def rcf_block(x, num_filters):
    # 간단한 RCF 블록: Convolution + BatchNorm + ReLU
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

# 입력 이미지의 차원을 정의합니다.
input_shape = (None, None, 3)  # 높이, 너비, 채널

# 입력 레이어를 정의합니다.
inputs = Input(shape=input_shape)

# RCF 네트워크를 구축합니다. (여기서는 단순화된 버전으로 2개의 RCF 블록만 사용)
x = rcf_block(inputs, 64)
x = rcf_block(x, 128)

# 마지막으로, 1개의 채널을 가진 컨볼루션 레이어를 추가합니다 (경계 탐지용).
outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

# 모델을 생성합니다.
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# 모델 요약을 출력합니다.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 64)    1792      
                                                                 
 batch_normalization (Batch  (None, None, None, 64)    256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, None, None, 64)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 128)   73856     
                                                                 
 batch_normalization_1 (Bat  (None, None, None, 128)   512       
 chNormalization)                                            

In [ ]:
import cv2
import numpy as np

# RCF 모델 로드 (사전 훈련된 모델 파일 필요)
rcf_model_path = 'path/to/rcf/model'  # 모델 파일 경로로 변경하세요.
rcf = cv2.dnn.readNet(rcf_model_path)

# 문서 이미지 로드
image_path = 'path/to/document/image.jpg'  # 이미지 파일 경로로 변경하세요.
image = cv2.imread(image_path)

# 이미지 전처리
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(500, 500),
                             mean=(104.00698793, 116.66876762, 122.67891434),
                             swapRB=False, crop=False)

# RCF로 경계 탐지
rcf.setInput(blob)
edges = rcf.forward()

# 경계 결과 처리
edges = np.squeeze(edges)  # 차원 축소
edges = (edges * 255).astype(np.uint8)  # 정규화 및 변환

# 경계 기반 문서 추출 및 변형 로직 (여기서는 단순히 결과를 보여줌)
cv2.imshow('Edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 참고: 실제 문서 스캔에서는 탐지된 경계를 기반으로 문서를 추출하고,
# 투시 변환(perspective transform)을 수행해야 합니다.